# Import packages

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import datetime
import holidays
import warnings
warnings.filterwarnings('ignore')

# Set Country holidays
us_holidays = holidays.US()

# Load Data

In [ ]:
#File path to the csv file
csv_file = "<path to>\ForecastingData.csv"

# Read csv file into dataframe
df = pd.read_csv(csv_file)

# Print first 5 rows in the dataframe
df.head()

In [ ]:
# OR CONNECT TO YOUR DATASET IN AZURE MACHINE LEARNING
#from azureml.core import Workspace, Dataset

#ws = Workspace.from_config()

#dataset = Dataset.get_by_name(ws, name='<enter dataset name from AML here>')
#df = dataset.to_pandas_dataframe()
#df.head()

# Filter by the store and item IDs to predict

In [ ]:
#Ids to Predict
ID1 = 2
ID2 = 1
#filter list to ids we want to predict
df = df.loc[(df['ID1'] == ID1) & (df['ID2'] == ID2)]

df.head()

# Create future 4 weeks for predictions

In [ ]:
#format time columns
#sort df by time
df['Time']=pd.to_datetime(df['Time'].astype(str))
df = df.sort_values(by='Time')

In [ ]:
#get last date from df

lastdate  = df['Time'].iloc[-1].date()
print(lastdate)

In [ ]:
#add next four week dates
#loop thru range of numbers by 7 up to 34 skip by 7
for days in range(7,34,7):
    #append new date to existing df
    df = df.append({'ID1' : ID1, 
                    'ID2' : ID2, 
                    'Time':(lastdate + datetime.timedelta(days=days)), 
                    'Value':0,
                    'RDPI': df['RDPI'].iloc[-1],
                    'IsUsNewYearsDay': False,
                    'IsUsLaborDay': False,
                    'IsBlackFriday': False,
                    'IsChristmasDay': False
                   } , ignore_index=True)

# Add Lag features for prev 26 weeks

In [ ]:
#add lag features
#grab the prev 26 weeks and add to current row
for index, item in df.iterrows():
    for shift in range(1,27):
        colName = 'Lag' + str(shift)
        df[colName] = df['Value'].shift(shift)

In [ ]:
predictionDf = df.tail(4).reset_index()
predictionDf = predictionDf.drop(columns='index')
predictionDf

# Create Time Features

In [ ]:
predictionDf['Year'] = pd.to_datetime(predictionDf['Time']).dt.year
predictionDf['Month'] = pd.to_datetime(predictionDf['Time']).dt.month
predictionDf['WeekOfMonth'] = pd.to_datetime(predictionDf['Time']).dt.day.apply(lambda day: math.ceil(int(day)/7))
predictionDf['WeekOfYear'] = pd.to_datetime(predictionDf['Time']).dt.week

In [ ]:
predictionDf[['Time','Year','WeekOfMonth','WeekOfYear']]

# Create Holiday columns Values

In [ ]:
def createHolidayFeatures(index):
    for days in range(0,7):
        newDate= predictionDf['Time'][index] + datetime.timedelta(days=days)
        holiday = us_holidays.get(newDate);
        if(holiday == "Thanksgiving"):
            predictionDf['IsBlackFriday'][index] = 1
        elif holiday == "Labor Day":
            predictionDf['IsUsLaborDay'][index] = 1
        elif holiday == "Christmas Day":
            predictionDf['IsChristmasDay'][index] = 1
        elif holiday == "New Year's Day":
            predictionDf['IsUsNewYearsDay'][index] = 1

In [ ]:
for index in range(len(predictionDf)):
    createHolidayFeatures(index)

In [ ]:
#print holidays
predictionDf[['Time','IsBlackFriday','IsUsLaborDay','IsChristmasDay','IsUsNewYearsDay']]

# Create Fourier Features

In [ ]:
def createFourierFeatures(index, weekofyear):
    #seasonality number weeks in year
    seasonality = 52
    #take index of data and mod seasonality. This gets the 0 to 51 weeks in a year
    #calcWeekOfYear = index % seasonality
    for s in range(1,5):
        cosColName = ('FreqCos' + str(s))
        sinColName = ('FreqSin' + str(s))
        #fix this its taking too long
        predictionDf[cosColName].loc[index] = math.cos(weekofyear*2*math.pi*s/seasonality)
        predictionDf[sinColName].loc[index] = math.sin(weekofyear*2*math.pi*s/seasonality)

In [ ]:
for index, weekofyear in predictionDf['WeekOfYear'].iteritems():
    createFourierFeatures(index,weekofyear)

In [ ]:
predictionDf[['FreqCos1', 'FreqSin1', 'FreqCos2', 'FreqSin2', 'FreqCos3', 'FreqSin3', 'FreqCos4', 'FreqSin4']]

In [ ]:
#Optional: graph fourier features
#sns.pointplot(x = 'Value', y = 'FreqCos1', data = df)
#sns.pointplot(x = 'Value', y = 'FreqSin1', data = df)

In [ ]:
#visualize the columns included in the df for debugging purposes
#list(df.columns) 

# Filter Data and Create Dictionary for API Post

In [ ]:
# Create Dictionary
predictionDf['Time'] = predictionDf['Time'].astype(str)
pre_dict = predictionDf.T.to_dict('dict')
#print(pre_dict)

# Create function to get prediction from API

In [ ]:

import urllib.request
import json
import os
import ssl

def getPrediction(index):

    #construct data object for post
    data = {
            "Inputs": {"input0":
                          [
                              pre_dict[index]
                          ],
                    },
                    "GlobalParameters":  {
                    }
            }
    #print(data)
    body = str.encode(json.dumps(data))
    
    url = '<endpoint>'
    api_key = '<key>' # Replace this with the API key for the web service
    headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}
    
    req = urllib.request.Request(url, body, headers)
    
    try:
        response = urllib.request.urlopen(req)
        result = response.read()
        return json.loads(result)
    except urllib.error.HTTPError as error:
        print(f'The request failed with status code: {str(error.code)} Error Info {error.info()}')
        print(json.loads(error.read().decode("utf8", 'ignore')))

# Get predictions

In [ ]:

for index in range(4): 
    predictionResult = getPrediction(index)
    #print(predictionResult)
    prediction = math.ceil(predictionResult['Results']['WebServiceOutput0'][0]['Forecast'])
    storeId = math.ceil(predictionResult['Results']['WebServiceOutput0'][0]['ID1'])
    itemId = math.ceil(predictionResult['Results']['WebServiceOutput0'][0]['ID2'])
    time = predictionDf['Time'][index]
    print(f'For week {time} item {itemId} the prediction is {prediction} items in store {storeId}')